In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import datetime, time

import modutils

data_dir = '../DataSets/Churn/'
#data_dir = '../DataSets/kkbox/churn/raw/'
logs_fmt = data_dir + 'user_logs/uldts_{0}.csv'
logm_fmt = data_dir + 'user_logs/uldtm_{0}.csv'
logu_fmt = data_dir + 'user_logs/uldtu_{0}.csv'
used_log = '201702'

In [5]:
fname = logs_fmt.format(used_log)
fname

'../DataSets/Churn/user_logs/uldts_201702.csv'

In [6]:
srclog = pd.read_csv(fname)

In [7]:
srcid = pd.DataFrame(np.array(srclog.groupby('msno').date.count().index), columns=['msno'])
srcid['uid'] = srcid.index
df_dates = pd.DataFrame(np.sort(np.array(srclog.date.value_counts().index)), columns=['date'])

srcid['fake'] = 0
df_dates['fake'] = 0

In [8]:
srcfull = pd.merge(srcid, df_dates, on='fake')[['msno', 'uid', 'date']]

In [9]:
#20 sec
srcfull = pd.merge(srcfull, srclog, how='left', on=['msno', 'date'])

In [10]:
srcfull['used'] = 1*(~srcfull.total_secs.isnull())
srcfull.num_25.view()[srcfull.used==0] = 0
srcfull.num_50.view()[srcfull.used==0] = 0
srcfull.num_75.view()[srcfull.used==0] = 0
srcfull.num_985.view()[srcfull.used==0] = 0
srcfull.num_100.view()[srcfull.used==0] = 0
srcfull.num_unq.view()[srcfull.used==0] = 0
srcfull.total_secs.view()[srcfull.used==0] = 0

In [11]:
srcfull['num_tot'] = srcfull.num_25 + srcfull.num_50 + srcfull.num_75 + srcfull.num_985 + srcfull.num_100
srcfull['pct_low'] = srcfull.num_25 / srcfull.num_tot
srcfull['pct_high'] = (srcfull.num_50 + srcfull.num_75 + srcfull.num_985) / srcfull.num_tot
srcfull['pct_100'] = (srcfull.num_100) / srcfull.num_tot
srcfull['pct_unq'] = (srcfull.num_unq) / srcfull.num_tot
srcfull['avg_sec'] = srcfull.total_secs / srcfull.num_tot

srcfull['ln_tot'] = np.log(1 + srcfull.num_tot)
srcfull['ln_sec'] = np.log(1 + srcfull.total_secs)
srcfull['avg_sec_n'] = (srcfull.avg_sec - 180) / 60

In [12]:
srcfull.pct_low.view()[srcfull.used==0]=0
srcfull.pct_high.view()[srcfull.used==0]=0
srcfull.pct_100.view()[srcfull.used==0]=0
srcfull.pct_unq.view()[srcfull.used==0]=0
srcfull.avg_sec.view()[srcfull.used==0]=0
srcfull.avg_sec_n.view()[srcfull.used==0]=0
srcfull.ln_tot.view()[srcfull.used==0]=0
srcfull.ln_sec.view()[srcfull.used==0]=0

In [13]:
srcfull['nxt_used'] = srcfull.used.shift(-1)
srcfull.nxt_used.view()[srcfull.uid.shift(-1)!=srcfull.uid] = np.nan

In [14]:
srcid[['msno','uid']].to_csv(logu_fmt.format(used_log), index=False)

In [34]:
flds = ['uid','date','used', 'nxt_used', 'ln_tot', 'ln_sec','pct_low','pct_high','pct_100','pct_unq','avg_sec_n']
len(flds), logm_fmt.format(used_log)

(11, '../DataSets/Churn/user_logs/uldtm_201702.csv')

In [ ]:
srcfull.to_csv(logm_fmt.format(used_log), index=False, columns=flds, float_format='%g')